# 13. Molecular dynamics simulations and analysis using Python

## Google Colab package installs

This installs the necessary packages for Google Colab. Please only run these if you are using Colab.

In [ ]:
# NBVAL_SKIP
!if [ -n "$COLAB_RELEASE_TAG" ]; then pip install condacolab; fi
import condacolab
condacolab.install()

In [ ]:
# NBVAL_SKIP
import condacolab
condacolab.check()
!mamba install -c conda-forge mdanalysis mdanalysistests openmm parmed nglview

In [ ]:
# NBVAL_SKIP
# enable third party jupyter widgets
from google.colab import output
output.enable_custom_widget_manager()

Later in the course you will look at how you can use Molecular Dynamics simulations as a powerful tool to explore the motion of complex systems.

To achieve this you will employ the Gromacs Molecular Dynamics engine, which is both very efficient and feature rich, but relies on the use of command-line calls rather than Python.

Here we demonstrate how you could use Python to both carry out and analyse Molecular Dynamics trajectories.

> Note: we intentionally do not go into the details of the Molecular Dynamics simulations as this will be covered in the Molecular Dynamics Workshop. This should be considered as more of a technical demonstration.

### Python libraries

In this tutorial we will be using three main non-standard python libraries:

1. [OpenMM](https://openmm.org/)
OpenMM is a high-performance toolkit for molecular simulations with a convenient Python interface.

2. [ParmEd](https://parmed.github.io/ParmEd/html/index.html)
A general tool for aiding the investigation of biomolecular systems using most popular molecular simulations packages.

2. [MDAnalysis](https://www.mdanalysis.org/):
MDAnalysis is a python library primarily developed to help with the anlysis of Molecular Dynamics (MD) trajectories. Beyond just MD, it offers many different tools and functions that can be useful when trying to explore atomistic models.

3. [NGLView](https://github.com/nglviewer/nglview)
NGLView is a powerful widget that allows you to visualise molecular models within jupyter notebooks.

4. [Matplotlib](https://matplotlib.org/)
One of the main plotting tools for python, matplotlib offers a wide range of functionality to generate graphs of everything from a simple scatter plot to [complex animated 3D plots](https://matplotlib.org/gallery/animation/random_walk.html#sphx-glr-gallery-animation-random-walk-py).

## 1. Simulating HIV-1 protease

Carrying on from the previous notebook, and in anticipation of the Docking workshop, we again look at the HIV-1 protease. Here we will be running a short simulation of the protein alone (without its ligand binder) in order to observe how the protein moves over time.

We have prepared ahead of time a version of the 1HSG system we looked at in the previous notebook with both the waters and ligand removed, and protons added. This can be found under `datafiles/1hsg.pdb`.

> Note: there are many ways to protonate system, here we used the [ProteinPlus](https://proteins.plus/) webserver from the ZBH Center for Bioinformatics.

#### A note on system conditions

In order to make sure that we can simulate the protein on your computer within a reasonable amount of time, we will use an _implicit solvent_ model. This means that we don't physically include water molecules, instead we alter the nonbonded interactions between the protein atoms in order to simulate the how these waters would influence these interactions.

#### Q1. How could this be problematic?

> Implicit waters do a good job at scaling interatomic interactions to account for the different dielectric constant which exist under solvated conditions. However, specific interactions between water and protein atoms are completely missed. This can be especially problematic when waters have a key structural interactions.
> Other issues include missing entropic effects related to water organisation within a system.

### 1.1 Creating an OpenMM simulation

Here we will use various OpenMM tools in order to create a simulation

In [ ]:
# First we load the PDB file into the OpenMM PDBFile class
from openmm.app import PDBFile, Modeller

pdb = PDBFile('./datafiles/1hsg.pdb')

# We pass this to a OpenMM modeller object which allows us to 
# edit and manipulate the protein system
model = Modeller(pdb.topology, pdb.positions)

In [ ]:
# Then we define a force field that will define the forces between the atoms of our protein
# Here we use the AMBER 14SB force field as defined by Maier et al.
# https://pubs.acs.org/doi/abs/10.1021/acs.jctc.5b00255
# We also select the GBn2 implicit solvent by Nguyen et al. https://pubmed.ncbi.nlm.nih.gov/25788871/
from openmm.app import ForceField

forcefield = ForceField('amber14/protein.ff14SB.xml', 'implicit/gbn2.xml')

In [ ]:
# Then we combine the forcefield and the protein model to create a "System"
# The OpenMM System defines all the necessary parameters needed to run a Molecular Dynamics Simulation
# We set our nonbonded interactions to "NoCutoff" which is necessary for implicit solvent simulations.
# This essentially means that interactions being all pairs of atoms are calculated.
# We also constraint our hydrogen bonds in order to use an integration timestep of 2 fs.
from openmm.app import NoCutoff, HBonds, GBn2
from openmm import unit

system = forcefield.createSystem(model.topology, nonbondedMethod=NoCutoff,
                                 constraints=HBonds)

In [ ]:
# Then we define an integrator which will integration the newtonian equations of motions for
# our protein system over time (with a discretisation of 2 fs, and a temperature of 298.15 K)
# Note we are using the  Langevin integrator
from openmm import LangevinIntegrator

integrator = LangevinIntegrator(
    298.15 * unit.kelvin, # The temperature we run our simulation at
    1.0 / unit.picoseconds, # The friction coefficient which controls our stochastic integrator
    2.0 * unit.femtoseconds, # The integration timestep
)

In [ ]:
# We choose what hardware we will run our simulation on
# Since the oxford biochemistry computer lab computers don't have GPUs, we default to CPUs
from openmm import Platform
platform = Platform.getPlatformByName('CPU')

In [ ]:
# Finally we create a simulation object which will run our simulation
from openmm.app import Simulation
simulation = Simulation(model.topology, system, integrator, platform)

# We set the initial coordinate of our simulation to those which we read in from the input PDB
simulation.context.setPositions(model.positions)

In [ ]:
# Next we minimize our system in order to resolve any large atomic clashes in our model
# Note: in practice you would carry out many more iterations of energy minimization
simulation.minimizeEnergy(maxIterations=500)

In [ ]:
# We set some reporters to write out the energies (StateDataReporter) and coordinates (NetCDFReporter) of the system
import sys
from parmed.openmm import StateDataReporter, NetCDFReporter

simulation.reporters.append(
    StateDataReporter(sys.stdout, 20, step=True, potentialEnergy=True,
                      kineticEnergy=True, temperature=True)
)
    
simulation.reporters.append(
    NetCDFReporter('1hsg.nc', 20, crds=True)
)

In [ ]:
# Now we run a short molecular dynamics simulation
simulation.step(1000)

In [ ]:
# Let's close our NetCDF reporter
simulation.reporters[1].finalize()

The simulation carried out here is only 2 picosecond in length in order to make sure it could be carried out on modest computers within the timeframe of this workshop. In practice, much much longer simulations (on the order of microseconds), would be carried out in order to obtain meaningful information about the protein dynamics.

## 2. Analyzing our simulation

Here we again use MDAnalysis and NGLView in order to examine our Molecular Dynamics trajectory.

First we create an MDAnalysis `Universe` using the OpenMM system and the `1hsg.nc` trajectory file we created above.

In [ ]:
import MDAnalysis as mda

universe = mda.Universe('datafiles/1hsg.pdb', '1hsg.nc')

Next we visualise the trajectory using NGLView

In [ ]:
import nglview

view = nglview.show_mdanalysis(universe)

By default this pre-sets the nglview to show the protein in the cartoon representation. Let's add a few options to change the display a little bit

In [ ]:
# Let's update the cartoon representation to colour the protein by secondary structure
view.update_cartoon(color='sstruc')

# Let's change the display a little bit
view.parameters = dict(camera_type='orthographic', clip_dist=0)

# Set the background colour to black
view.background = 'black'

# Call protein_view to visualise the trajectory
view

As you can be seen from the trajectory, the HIV-1 protease structure does indeed move, but by how much? In the next section we will see how we can use MDAnalysis to quantify backbone fluctuations.

### Calculating the root-mean-square deviation

In order to gain a quantitative description of how the HIV-1 protease moves in our simulation we can calculate the root-mean-square deviation (RMSD) of the protein backbone.

The RMSD gives us an idea of how 'stable' our protein is when compared to our starting, static, structure. The lower the RMSD is the, more stable we can say our protein is.

The RMSD as a function of time, $\rho (t)$, can be defined by the following equation:

\begin{equation}
\rho (t) = \sqrt{\frac{1}{N}\sum^N_{i=1}w_i\big(\mathbf{x}_i(t) - \mathbf{x}^{\text{ref}}_i\big)^2}
\end{equation}

Luckily MDAnalysis has its own built-in function to calcualte this, we can import it like we did before.

In [ ]:
from MDAnalysis.analysis.rms import RMSD as rmsd

In order to calculate the RMSD for every frame in our trajectory we will need:

* A universe object
* A selection of atoms


Our universe object will be the input universe we created above and for our selection we will use the backbone atoms.

In [ ]:
# Run our RMSD calculation

R = rmsd(universe,
         select='backbone')
R.run(verbose=True)

We can easily plot this data using the common matplotlib plotting package.

In [ ]:
import matplotlib.pyplot as plt

plt.plot(R.results.rmsd.T[0], R.results.rmsd.T[2])
plt.ylabel(r'RMSD ($\AA$)')
plt.xlabel('Frame number')

### Calculating the root-mean-square deviation

To look at how each residue flucuates over it's average postion we can use the closely related measurement of root-mean-square fluctuation (RMSF).

The RMSF for an atom,  $\rho_i$, is given by:

\begin{equation}
\rho_i = \sqrt{\sum^N_{i=1} \big\langle(\mathbf{x}_i - \langle \mathbf{x}_i \rangle )^2 \big\rangle }
\end{equation}


In [ ]:
from MDAnalysis.analysis import rms, align

To begin with, we have to align our trajectory to remove any fluctuations related to rotation and translational motion

In [ ]:
# We can generate an average structure to align to with the `align.AligntStructure` class.

average = align.AverageStructure(universe, universe, select='protein and name CA',
                                 ref_frame=0).run()
ref = average.results.universe

In [ ]:
# Next we align our trajectory against this average structure

aligner = align.AlignTraj(universe, ref,
                          select='protein and name CA',
                          in_memory=True).run()

In [ ]:
# Finally we calculate the RMSF based on the alpha carbons of our protein
c_alphas = universe.select_atoms('protein and name CA')
R = rms.RMSF(c_alphas).run()

In [ ]:
# We can plot our results using matplotlib
plt.plot(c_alphas.resids, R.results.rmsf)
plt.xlabel('Residue number')
plt.ylabel('RMSF ($\AA$)')
plt.legend();

We can visualise the fluctuations using NGLview by passing the values to the `tempfactor` attribute of the system.

In [ ]:
universe.add_TopologyAttr('tempfactors') # add empty attribute for all atoms
protein = universe.select_atoms('protein') # select protein atoms
for residue, r_value in zip(protein.residues, R.rmsf):
    residue.atoms.tempfactors = r_value

In [ ]:
view = nglview.show_mdanalysis(universe)
view.update_representation(color_scheme='bfactor')
view

> Answer: the PDB page states the number of protein residues, so there are 128 non-protein residues

> Answer: Here we go from red being low beta factor regions, to blue being high ones. That is to say that bluer regions are more mobile.

> Answer: think about which areas are more solvent exposed and therefore more likely to be in motion.

> Answer: the loops composed of residues 49-52 are quite mobile and close to the ligand. In fact previous [work by Hornak et al.](https://www.pnas.org/content/103/4/915) shows that these can spontaneously open and close. Doing a molecular dynamics simulation, might be helpful in elucidating how these loops move.